# Paddle2ONNX Technology Walkthrough
这篇文章简要的介绍 [Paddle2ONNX](https://github.com/PaddlePaddle/Paddle2ONNX) 背后的技术， 希望能对理解 Paddle2ONNX 项目有一定帮助。

## 回顾一下 Paddle2ONNX 是做什么的
Pddle2ONNX 项目的定位是把一个 PaddlePaddle 格式的模型转换成一个 ONNX 格式的模型。 为了能更清晰的展示，它在做什么，我们先准备一个简单的 PaddlePaddle 的模型。

In [1]:
import paddle
import paddle.nn.functional as F

class MyModel(paddle.nn.Layer):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.linear1 = paddle.nn.Linear(input_size, hidden_size)
        self.linear2 = paddle.nn.Linear(hidden_size, 1)

    def forward(self, inputs):
        x = self.linear1(inputs)
        x = F.sigmoid(x)
        x = self.linear2(x)
        return x

input_size, hidden_size = 8, 4
model = MyModel(input_size, hidden_size)

x_input_spec = paddle.static.InputSpec([None, input_size], 'float32', 'x')
paddle.jit.save(model, "./demo", input_spec=[x_input_spec])

I0625 16:23:07.914927 84493696 kernel_dispatch.h:102] Get BackendSet from tensor
I0625 16:23:07.919880 84493696 kernel_dispatch.h:102] Get BackendSet from tensor
I0625 16:23:08.120766 84493696 program_interpreter.cc:243] New Executor is Running.


上面的代码将一个 PaddlePaddle 的模型存储了下来。 请注意，这里只是为了示意，所以忽略了模型的训练的部分。 接下来，我们用 Paddle2ONNX 把 PaddlePaddle 的模型转换成 ONNX 格式的模型。

In [2]:
import paddle2onnx

paddle2onnx.export(
        model_file='./demo.pdmodel',
        params_file='./demo.pdiparams',
        save_file='demo.onnx'
        )


[Paddle2ONNX] Start to parse PaddlePaddle model...
[Paddle2ONNX] Model file path: ./demo.pdmodel
[Paddle2ONNX] Parameters file path: ./demo.pdiparams
[Paddle2ONNX] Start to parsing Paddle model...
[Paddle2ONNX] Use opset_version = 11 for ONNX export.
[Paddle2ONNX] PaddlePaddle model is exported as ONNX format now.


运行完上面的代码后，会多出来一个 demo.onnx 的文件，这就是转换后的 ONNX 格式的模型文件。 BTW： 你也可以用 `paddle2onnx` 这个命令行工具来做到同样的事情，跟上面的代码本质上是一样的。 到此为止，硬盘上会多出来的这 4 个文件。
- demo.pdmodel ：  这个文件存储的是 PaddlePaddle 模型的模型结构
- demo.pdiparams ： 这个文件存储的是 PaddlePaddle 模型的模型参数
- demo.pdiparams.info ： 这个文件存储的是在使用模型 Fintune 时需要的额外的信息， 对于 Paddle2ONNX 项目来说，可以忽略。
- demo.onnx ： 这个文件存储的是 ONNX 格式的模型（包括了模型结构和模型参数）。

所以， Paddle2ONNX 所做的事情，就是把 demo.pdmodel + demo.pdiparams 转换成 demo.onnx 。

BTW： Paddle2ONNX 项目，从 ONNX 的视角，是一个 [Converter](https://onnx.ai/onnx/intro/converters.html)。

## 剥开这些模型文件看看

我们可以使用如 [netron](https://netron.app/) 这样的工具来可视化出来 PaddlePaddle 模型和 ONNX 模型。 也可以使用代码加载文件然后看看这些模型文件里具体存储了哪些信息。

### 我们先剥开 demo.pdmodel 文件看看。


In [3]:
import paddle.base.proto.framework_pb2 as pppb

prog = pppb.ProgramDesc()
with open('./demo.pdmodel', "rb") as f:
    prog.ParseFromString(f.read())
    
print(str(prog)[:100] + '\n...\n' + str(prog)[-100:])
# print(prog)

blocks {
  idx: 0
  parent_idx: -1
  vars {
    name: "linear_0.b_0"
    type {
      type: LOD_TENS
...
sion: 1
    }
  }
  pair {
    op_name: "affine_grid"
    op_version {
      version: 1
    }
  }
}



demo.pdmodel 文件的本质是一个根据 [framework.proto](https://github.com/PaddlePaddle/Paddle/blob/develop/paddle/fluid/framework/framework.proto) 的定义，使用 [protobuf](https://github.com/protocolbuffers/protobuf) 所自动生成的 `paddle.base.proto.framework_pb2`  进行解析，操作的持久化存储下来的模型结构信息。 因为 protobuf 所提供的能力， 使用其他编程语言，也同样可以解析和操作这个文件。 Paddle2ONNX 里，是使用 C++ 来解析这个文件， 具体进行解析这个文件的代码是：[PaddleParser](https://github.com/PaddlePaddle/Paddle2ONNX/blob/develop/paddle2onnx/parser/parser.cc) 。 

[framework.proto](https://github.com/PaddlePaddle/Paddle/blob/develop/paddle/fluid/framework/framework.proto) 文件，可以理解为定义 PaddlePaddle 模型结构的 Schema， 所以 Paddle2ONNX 里也存储了一份， 是 [p2o_paddle.proto](https://github.com/PaddlePaddle/Paddle2ONNX/blob/develop/paddle2onnx/proto/p2o_paddle.proto) 。 这两个 proto 文件，除了 package name 外，其他部分是一模一样的。

如果你有兴趣， 可以用 `paddle.base.proto.framework_pb2` 编辑，修改这个模型结构。你也可以用 [framework.proto](https://github.com/PaddlePaddle/Paddle/blob/develop/paddle/fluid/framework/framework.proto) 文件， 使用 protobuf，构建出来完全不需要 PaddlePaddle 和 Paddle2ONNX 的解析 PaddlePaddle 模型结构的工具。

Some Fun: PaddlePaddle 模型里， 存储了 op_callstack， 那里记录了导出模型的机器上的文件路径，所以社区里会有[这样的 issue](https://github.com/PaddlePaddle/PaddleOCR/issues/11755)。

### 剥开 demo.onnx 看看

In [4]:
import onnx

m = onnx.load('./demo.onnx')
print(str(m)[:100] + '\n...\n' + str(m)[-100:])
# print(m)

ir_version: 6
graph {
  node {
    output: "linear_0.b_0"
    op_type: "Constant"
    attribute {
  
...
 dim_value: 1
          }
        }
      }
    }
  }
}
opset_import {
  domain: ""
  version: 11
}



没错， demo.onnx 也是一个使用 protobuf 进行持久化存储下来的文件。 ONNX 项目规范了这个模型格式，让深度学习模型可以在不同的框架和引擎之间进行交换。 除了存储了模型结构之外， ONNX 格式的文件里也存储了模型的权重， 而 PaddlePaddle 的模型权重文件，是在 demo.pdiparams 文件里存储的。

### 剥开 demo.pdiparams 看看

PaddlePaddle 模型权重是用 PaddlePaddle 所定义的格式，以二进制的方式存储起来的。 但是存储格式，并没有详细的文档。只能通过阅读 Paddle2ONNX 的源码 [PaddleParser::LoadParams](https://github.com/PaddlePaddle/Paddle2ONNX/blob/develop/paddle2onnx/parser/parser.cc) 来了解其格式。 下面的代码， 用 python 实现了类似的功能。

In [5]:
import struct 
import numpy as np

with open('./demo.pdiparams', 'rb') as f:
    raw_content = f.read()

idx = 0
while(idx < len(raw_content)):
    magic_number1, lod_level, magic_number_2, tensor_desc_size = struct.unpack('=IQIi', raw_content[idx:idx+20])
    print(f"lod_level: {lod_level} tensor_desc_size {tensor_desc_size}")
    idx = idx + 20

    tensor_desc = pppb.VarType.TensorDesc()
    tensor_desc.ParseFromString(raw_content[idx:idx+tensor_desc_size])
    idx = idx+tensor_desc_size
    
    numel = 1
    for ele in tensor_desc.dims:
        numel = numel * ele
    
    # 4 in next line because sizeof(float32)=4
    weight = np.frombuffer(raw_content[idx:idx+numel*4], dtype=np.float32)
    print(f"shape: {tensor_desc.dims}")
    print(f"weight: {weight}")
    idx = idx + numel * 4

lod_level: 0 tensor_desc_size 4
shape: [4]
weight: [0. 0. 0. 0.]
lod_level: 0 tensor_desc_size 6
shape: [8, 4]
weight: [ 0.56069726  0.03711593  0.02164149 -0.03126097 -0.5148256  -0.59038967
 -0.49709237  0.58931655 -0.2892736   0.24474019 -0.02178019  0.58994645
 -0.01471817  0.6165579   0.52084273 -0.47981268 -0.5484108   0.30529386
 -0.49860376  0.256252    0.24301541  0.32295316  0.46775275 -0.14265221
  0.10271853 -0.65563273 -0.608733   -0.5193291   0.20356852 -0.3251919
  0.22648627 -0.02558661]
lod_level: 0 tensor_desc_size 4
shape: [1]
weight: [0.]
lod_level: 0 tensor_desc_size 6
shape: [4, 1]
weight: [-0.16669196  0.75233805 -1.092682   -0.9836202 ]


接下来，通过直接打印出来模型里的参数，我们可以验证出来，所存储在文件里的权重，跟直接打印出的模型参数是一样的。

In [6]:
print(model.linear1.bias)
print(model.linear1.weight)
print(model.linear2.bias)
print(model.linear2.weight)

Parameter containing:
Tensor(shape=[4], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0., 0., 0., 0.])
Parameter containing:
Tensor(shape=[8, 4], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[ 0.56069726,  0.03711593,  0.02164149, -0.03126097],
        [-0.51482558, -0.59038967, -0.49709237,  0.58931655],
        [-0.28927359,  0.24474019, -0.02178019,  0.58994645],
        [-0.01471817,  0.61655790,  0.52084273, -0.47981268],
        [-0.54841077,  0.30529386, -0.49860376,  0.25625199],
        [ 0.24301541,  0.32295316,  0.46775275, -0.14265221],
        [ 0.10271853, -0.65563273, -0.60873300, -0.51932907],
        [ 0.20356852, -0.32519189,  0.22648627, -0.02558661]])
Parameter containing:
Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.])
Parameter containing:
Tensor(shape=[4, 1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[-0.16669196],
        [ 0.75233805],
        [-1.09268200],
        [-0.98

一些额外的说明：

- 所存储的模型权重跟模型结构里的 Variable 的对应关系： 如果查看 `prog.blocks[0].vars` 的话，你会发现，其数量是大于存储的 4 个模型权重的，因为 `vars` 还包括了其他的参与计算的 Variable。 他们对应的关系的建立的逻辑是按照 `prog.blocks[0].vars` 的顺序，挑选出被认定为模型权重的 Variable 。这部分在 Paddle2ONNX 里的实现是在 [PaddleParser::GetParamNames](https://github.com/PaddlePaddle/Paddle2ONNX/blob/develop/paddle2onnx/parser/parser.cc)。 坦白说，如果能在存储模型时建立起来权重跟 Variable 的对应关系，会更容易理解。这方面， ONNX 的模型文件就非常容易理解。
- LoD 是什么： LoD 是 Level of Details 的缩写，这是飞桨最开始设计里的一个特色，可以方便的表达不定长的 tensor 组成的 batch， 跟 Tensorflow 里的 [Ragged Tensor](https://www.tensorflow.org/guide/ragged_tensor)，或者 pytorch 里的 [nested tensor](https://pytorch.org/docs/stable/nested.html) 是类似的。 Paddle2ONNX 目前只能处理 `lod_level = 0` 的情况，当 `lod_level != 0` 时，会直接解析失败。
- magic＿number1, magic_number2 是什么： 这是存储在模型权重文件里的两个神秘数字（版本？）， Paddle2ONNX 里用不到。
- 权重的精度：模型权重文件里读到的 tensor_desc 里记录了数据类型（FP32， FP16， 等等）。
- block 又是什么：如果你把飞桨的模型结构当成一段计算机程序，那么每个 block 可以理解为一段程序代码。 当程序中有条件判断时，就会有多个 block。 现在， Paddle2ONNX 只能处理 Program 中仅有 1 个 block 的情况。

In [7]:
print('>>> all variables:')
print('\n'.join([f"{x.name}, {x.persistable}" for x in prog.blocks[0].vars]))
print('>>> all ops:')
print('\n'.join([op.type for op in prog.blocks[0].ops]))

>>> all variables:
linear_0.b_0, True
linear_0.tmp_0, False
linear_0.tmp_1, False
linear_0.w_0, True
linear_1.b_0, True
linear_1.tmp_0, False
linear_1.tmp_1, False
linear_1.w_0, True
save_infer_model/scale_0.tmp_0, False
sigmoid_0.tmp_0, False
x, False
feed, True
fetch, True
>>> all ops:
feed
matmul_v2
elementwise_add
sigmoid
matmul_v2
elementwise_add
scale
fetch


## 小讨论

本文前面的部分，我们把 PaddlePaddle 模型的格式跟 ONNX 模型的格式都概要的通过代码的形式展示了出来。按照一般的做格式转换的项目的基本流程（例如，把 jpg 文件转换成 png 文件），基本的做法，应该就是先解析源格式的文件，然后根据解析得到的信息，用目标格式重新构建文件，并存储。看起来并不复杂，就这篇文章里作为示例用的 demo 模型来说，再稍进一步，已经可以用 python 构建出来一个 ONNX 模型了。 但对于要能把所有 PaddlePaddle 模型转换成 ONNX 模型格式的 Paddle2ONNX 项目来说，面临着如下的重要挑战：

1. 模型格式的演进： 首先是 PaddlePaddle 模型格式的演进，新的 PaddlePaddle 的版本的发布，都潜在的会有新的模型格式的变化，即将发布的飞桨 3.0 版本， 因为 PIR 的正式上线，会带来不少的变化。这对于 Paddle2ONNX 项目来说，是需要迭代支持的。 其次是 ONNX 模型格式的演进， 每年发布 3~5 个 release 的 ONNX 的模型格式也会有新的变化。 作为一个 Converter 的 Paddle2ONNX 需要能够跟上节奏（keep up the rhythm）。
1. OP Coverage ： PaddlePaddle 模型里的每一个 OP 都需要语义正确的，转换为 ONNX 格式里的表示，这是一个相当大的工作量。
1. 控制流，不规则张量，混合精度计算，模型优化，自定义OP，量化，等等情况的支持。


BTW：Paddle2ONNX 项目曾经是完全用 python 来实现的， 例如： [v0.9](https://github.com/PaddlePaddle/Paddle2ONNX/tree/v0.9)。 但是出于能够更好地解决边界情况的原因，项目组后来重构成了用 C++ 作为其核心来实现。


## 构建 ONNX 模型

ONNX 对于深度学习模型的抽象比较容易理解。就是一个由节点（Node），构成的图（Graph）。图有输入（Input）和输出（Output）。每个节点（Node）也有输入（Input）和输出（Output），以及代表对输入进行计算得到输出的计算方式的算子（Operator），和固定的属性（Attribute）。更详细的介绍可以参考[ONNX Concept](https://onnx.ai/onnx/intro/concepts.html)。

下面是构建出来跟使用 paddle2onnx 转换出来的 `demo.onnx` 等价的 ONNX 模型的代码。

In [8]:
from onnx import TensorProto
from onnx.helper import (
        make_model, make_node, make_graph,
        make_tensor_value_info)
from onnx.checker import check_model

x = make_tensor_value_info('x', TensorProto.FLOAT, [None, input_size])
y = make_tensor_value_info('y', TensorProto.FLOAT, [None])

weights = {'A':model.linear1.weight,
        'A_bias':model.linear1.bias,
        'B':model.linear2.weight,
        'B_bias':model.linear2.bias}

weight_nodes = []

for name,weight in weights.items():
    n =  make_node(
            "Constant", [], [name],
            value=onnx.helper.make_tensor(
                name=name,
                data_type=onnx.TensorProto.FLOAT,
                dims=weight.shape,
                vals=weight.numpy().flatten(),
                )
            )
    weight_nodes.append(n)

node1 = make_node('MatMul', ['x', 'A'], ['XA'])
node2 = make_node('Add', ['XA', 'A_bias'], ['linear1_out'])
node3 = make_node('Sigmoid', ['linear1_out'], ['sigmoid'])
node4 = make_node('MatMul', ['sigmoid', 'B'], ['sigmoidB'])
node5 = make_node('Add', ['sigmoidB', 'B_bias'], ['y'])

graph = make_graph(weight_nodes + [node1, node2, node3, node4, node5],  # nodes
                    'demograph',  # a name
                    [x],  # inputs
                    [y])  # outputs

onnx_model = make_model(graph)
check_model(onnx_model)
print(onnx.helper.printable_graph(onnx_model.graph))

graph demograph (
  %x[FLOAT, ?x8]
) {
  %A = Constant[value = <Tensor>]()
  %A_bias = Constant[value = <Tensor>]()
  %B = Constant[value = <Tensor>]()
  %B_bias = Constant[value = <Tensor>]()
  %XA = MatMul(%x, %A)
  %linear1_out = Add(%XA, %A_bias)
  %sigmoid = Sigmoid(%linear1_out)
  %sigmoidB = MatMul(%sigmoid, %B)
  %y = Add(%sigmoidB, %B_bias)
  return %y
}


飞桨模型的结构里尽管有 Program、Block、Variable、Operator 等等的概念，但其本质上也是一个图（Graph）。构建 ONNX 模型的关键在于基于从飞桨模型当中找到的参数，算子，输入输出，然后使用 ONNX 提供的 API 构建出来与飞桨模型等价的图（Graph）。
在 Paddle2ONNX 项目中，构建 ONNX 模型的代码的实现在这里： [ModelExporter::Run](https://github.com/PaddlePaddle/Paddle2ONNX/blob/develop/paddle2onnx/mapper/exporter.cc)。我们打印出来使用 paddl2onnx 转换出来的示例的模型结构可以看到，跟上面的用 python 构建的 ONNX 的图是一样的。

In [9]:
print('>>> onnx graph converted using paddle2onnx :')
print(onnx.helper.printable_graph(m.graph))

>>> onnx graph converted using paddle2onnx :
graph Model from PaddlePaddle. (
  %x[FLOAT, p2o.DynamicDimension.0x8]
) {
  %linear_0.b_0 = Constant[value = <Tensor>]()
  %linear_0.w_0 = Constant[value = <Tensor>]()
  %linear_1.b_0 = Constant[value = <Tensor>]()
  %linear_1.w_0 = Constant[value = <Tensor>]()
  %p2o.MatMul.1 = MatMul(%x, %linear_0.w_0)
  %p2o.Add.1 = Add(%p2o.MatMul.1, %linear_0.b_0)
  %sigmoid_0.tmp_0 = Sigmoid(%p2o.Add.1)
  %p2o.MatMul.3 = MatMul(%sigmoid_0.tmp_0, %linear_1.w_0)
  %save_infer_model/scale_0.tmp_0 = Add(%p2o.MatMul.3, %linear_1.b_0)
  return %save_infer_model/scale_0.tmp_0
}


## OP Mapper

构建 ONNX 模型中的输入，输出和参数（weight），对于每个模型来说，都是固定的转换流程。对于 Paddle2ONNX 来说，工作量最大的部分在于如何把飞桨模型里的算子，对应到等价的 ONNX 算子里的一个或多个算子的组合上，确保语义是等价的，计算结果是能匹配上的。Paddle2ONNX 已经搭建好了脚手架来更方便的做这个工作。当需要新增一个算子支持时，只需要关注添加 \对应的 OP Mapper 就可以了。

以示例里的模型里的 `matmul_v2` 这个飞桨的算子为例，它会经过 `MatmulV2Mapper` 来转换成 `MatMul` 和 `Identity` 两个算子。（其中  `Identity` 这个算子是因为做自动的精度转换而添加的）。关于实际的添加 OP Mapper 的方法，可以参看文档：[Paddle2ONNX_Development_Guide.md](https://github.com/PaddlePaddle/Paddle2ONNX/blob/develop/docs/zh/Paddle2ONNX_Development_Guide.md)，本文就不再赘述了。


BTW：你可以用 [MapperHelper::GetAllOps](https://github.com/PaddlePaddle/Paddle2ONNX/blob/develop/paddle2onnx/mapper/register_mapper.h)，来导出，所有已经支持的飞桨算子的列表。（需要找时间，把这个 API 导出到 python 端，会更加方便一些）。

## Debug Tricks & Misc
- 建议使用 `pip install -e .` 来以 editable mode 在本地编译和开发 Paddle2ONNX，这样可以比较方便的进行迭代开发。
- 可以在调试时使用 `P2OLogger` 来打印日志，来观察实际的运行过程。
- cmake 里有一个编译选项 `PADDLE2ONNX_DEBUG`，目前只起到了非常小的作用，也许需要有人能真正把这个编译选项下的日志等等规范起来，来方便 Paddle2ONNX 的开发。
- 代码仓库里目前还没有真正利用上 linting 等工具，也许需要有人来投入精力做一下。

## Ending

这篇文章是基于粗浅的对 Paddle2ONNX 的理解完成的，还很粗浅，希望对开发 Paddle2ONNX 能有帮助。